In [13]:
!pip install transformers pandas

In [20]:
import pandas as pd

# Replace with your CSV path
df = pd.read_csv('synthetic_hourly_sheet_data_3000.csv')
df.head()


,Date,Shift,Machine/Workstation ID,Operator Name,Product Name / Part Number,Target Output,Actual Output,Cumulative Output,Defects/Rework Units,Downtime (Minutes),Reason for Downtime,Operator Remarks
0,2025-04-14,Morning,M101,Meera,P001,40,30,60,3,0,NaN,Check sensor
1,2025-04-02,Afternoon,M102,Raj,P003,60,53,53,4,0,NaN,Check sensor
2,2025-04-07,Night,M102,Meera,P003,43,43,86,0,21,Material shortage,Check sensor
3,2025-04-01,Night,M101,Raj,P003,45,43,86,3,22,NaN,NaN
4,2025-04-15,Morning,M102,Anita,P001,49,44,132,5,0,NaN,NaN


In [21]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Use text-generation pipeline instead of conversational
# Load the DialoGPT model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Create the text-generation pipeline
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [22]:
def process_query(query):
    query = query.lower()

    if "defects" in query and "today" in query:
        today = df['Date'].max()  # or use actual date logic
        defects_today = df[df['Date'] == today]['Defects/Rework Units'].sum()
        return f"Total defects reported today ({today}) is {defects_today}"

    elif "operator" in query and "produce" in query:
        for op in df['Operator Name'].unique():
            if op.lower() in query:
                # Check the actual column name in your CSV
                # and replace 'Actual Output (Produced Units per Hour)' with it
                # For example, if the column is named 'Actual Output', use:
                total_units = df[df['Operator Name'] == op]['Actual Output'].sum()
                return f"Operator {op} produced a total of {total_units} units."

    return "Sorry, I couldn't understand the question."

# Example
print(process_query("How many units did operator Raj produce?"))

Operator Raj produced a total of 32877 units.


In [26]:
def process_query(query):
    query = query.lower()

    # Normalize date
    today = df['Date'].max()

    # --- Handle defect-related queries ---
    if "defect" in query:
        # Check for 'today' context
        data = df[df['Date'] == today] if "today" in query else df

        # Check for specific operator
        for op in df['Operator Name'].unique():
            if op.lower() in query:
                total = data[data['Operator Name'] == op]['Defects/Rework Units'].sum()
                return f"Operator {op} reported {total} defects {'today' if 'today' in query else ''}."

        # Check for specific machine
        for machine in df['Machine/Workstation ID'].unique():
            if machine.lower() in query:
                total = data[data['Machine/Workstation ID'] == machine]['Defects/Rework Units'].sum()
                return f"Machine {machine} reported {total} defects {'today' if 'today' in query else ''}."

        # If no operator or machine specified
        total = data['Defects/Rework Units'].sum()
        return f"Total defects {'today' if 'today' in query else ''}: {total}"

    # --- Handle production queries ---
    if "produce" in query or "output" in query:
        for op in df['Operator Name'].unique():
            if op.lower() in query:
                total_units = df[df['Operator Name'] == op]['Actual Output'].sum()
                return f"Operator {op} produced {total_units} units in total."

    return "Sorry, I couldn't understand the question."


In [ ]:
import pandas as pd

# Load your CSV file
df = pd.read_csv('synthetic_hourly_sheet_data_3000.csv')

# Convert date to datetime (optional but helpful)
df['Date'] = pd.to_datetime(df['Date'])

# Query processor function
def process_query(query):
    query = query.lower()
    today = df['Date'].max()

    # --- Handle defect-related queries ---
    if "defect" in query:
        data = df[df['Date'] == today] if "today" in query else df

        for op in df['Operator Name'].unique():
            if op.lower() in query:
                total = data[data['Operator Name'] == op]['Defects/Rework Units'].sum()
                return f"Operator {op} reported {total} defects {'today' if 'today' in query else ''}."

        for machine in df['Machine/Workstation ID'].unique():
            if machine.lower() in query:
                total = data[data['Machine/Workstation ID'] == machine]['Defects/Rework Units'].sum()
                return f"Machine {machine} reported {total} defects {'today' if 'today' in query else ''}."

        total = data['Defects/Rework Units'].sum()
        return f"Total defects {'today' if 'today' in query else ''}: {total}"

    # --- Handle production/output-related queries ---
    if "produce" in query or "output" in query:
        for op in df['Operator Name'].unique():
            if op.lower() in query:
                total_units = df[df['Operator Name'] == op]['Actual Output'].sum()
                return f"Operator {op} produced {total_units} units in total."

    return "Sorry, I couldn't understand the question."

# ✅ Input Loop
while True:
    user_input = input("Ask a question (or type 'exit'): ")
    if user_input.lower() == "exit":
        print("Exiting chatbot. Goodbye!")
        break
    response = process_query(user_input)
    print(response)


Ask a question (or type 'exit'): Show today's total defect by operator Meera
Operator Meera reported 109 defects today.
Ask a question (or type 'exit'): How many units did Meera produce?
Operator Meera produced 32142 units in total.
Ask a question (or type 'exit'): Show today's total defects by Machine M102
Machine M102 reported 150 defects today.
Ask a question (or type 'exit'): How many units did Raj produce today?
Operator Raj produced 32877 units in total.
Ask a question (or type 'exit'): Show today's defects by Machine M103
Machine M103 reported 163 defects today.
